In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
import chromadb
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from uuid import uuid4
from langchain_core.documents import Document
from langchain_ollama import OllamaLLM
from langchain import PromptTemplate, LLMChain
from langchain_ollama import OllamaLLM

In [3]:
loader= PyPDFDirectoryLoader("pdfs")

In [4]:
 
data=loader.load()

In [10]:
data

[Document(metadata={'source': 'pdfs\\TheHistoryofApple1.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/364326882\nThe History of Apple\nArticle · October 2022\nCITATIONS\n0\nREADS\n29,507\n2 authors, including:\nOdina Kamolova\nWestminster International University in Tashkent\n4 PUBLICATIONS\xa0\xa0\xa01 CITATION\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Odina Kamolova on 14 October 2022.\nThe user has requested enhancement of the downloaded file.'),
 Document(metadata={'source': 'pdfs\\TheHistoryofApple1.pdf', 'page': 1}, page_content='IJESC, September 2022                                                                        29800                              http:// ijesc.org/ \n \nISSN 2321 3361 © 2022 IJESC  \n  \n \n \nThe History of Apple \nMuxtorova Nasiba1, Kamolova Odina2 \nUniveristy of Westminster, \nTASHKENT Uzbekistan. \n \nIntroduction:    \nApp

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=15)
text_chunk = text_splitter.split_documents(data)

In [12]:
emdedding= OllamaEmbeddings(model="snowflake-arctic-embed:latest " )

In [13]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=emdedding,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [14]:
#adding data into the database
uuids = [str(uuid4()) for _ in range(len(text_chunk))]
 
vector_store.add_documents(documents=text_chunk, ids=uuids)

['76f9e109-d73a-4132-98ae-ec13b791cb47',
 '660e1249-394a-4391-bb38-9ec42adcffc5',
 'f67dcce9-1b8a-4c8d-ace5-384dd4dfd5f8',
 '2b9c0952-9c11-4b23-896d-99f3583fead5',
 'cc676280-1408-4a48-bcb3-a0e0007829d9',
 'dc763746-ef95-41a3-ad0a-e7d569747127',
 'c642232b-80c6-4f37-b242-1e4ee3a91684',
 '0038fb81-7031-47f5-b22a-3efdf00635ac',
 '1e06d07f-4cb5-47ea-806e-1ae3389712b0',
 'b8b71f14-7f27-4dd0-97a4-c37520b8a853',
 'c7a8ffdb-6f84-48b1-bdea-311e685d766b',
 'a3cefed1-8b4b-4236-9f86-b81c090aace3',
 '0fd9bca6-5999-4c98-b35c-89969f22be05',
 '2fbbb616-2968-4208-a5f8-f852d58dabae',
 'df97a909-03a1-4e46-a2cf-eab0b51fe1ac',
 '1f217731-9eb4-4d0d-bd88-0a16968b5aa1',
 'fe9f7275-e426-4c76-961b-7a1b8db8db3b',
 '135cfc34-389a-4b01-8c16-ab817f26c651',
 '31b6d6c8-e779-4fa0-9cd3-6afb2a061586',
 '45bee964-ad24-4bed-9fbd-dbb7b6c9addf',
 'c7d9ec5d-85a6-4733-bf66-26eb08ddb775',
 'f2749b0d-d37e-40ee-a56a-34a0bae35c47',
 'b10c21ca-6e50-4de7-8fc9-b4e47495987c',
 '7762e3d0-7550-4cfb-994f-07b1a9ba992a',
 '11bc0f79-e73d-

In [26]:
#llm model loaded
llm=OllamaLLM(model="stablelm-zephyr:3b")

In [42]:
#templete for the llm model
template = """
You are an expert assistant specializing in Apple's history. Provide accurate, concise, and contextually relevant answers based solely on the information provided. Do not speculate or give opinions. Avoid referencing the context directly or acknowledging its existence. Simply respond to the user query with a direct and informative answer.

### Context:
{context}

### User Query:
{query}

### Your Response:

"""

In [43]:
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template,
)

In [44]:
apple_history_chain = LLMChain(llm=llm, prompt=prompt)

In [46]:
while True:
    user_query = input("Ask a question about Apple's history (type 'exit' to quit): ")
    
    if user_query.lower() == "exit":
        print("Goodbye!")
        break
    
    # Fetch relevant context using the context function
    results = vector_store.similarity_search(user_query, k=2)
    
    # Run the chain with the given context and query
    response = apple_history_chain.run(llm=llm, context=results, query=user_query)
    print("\nResponse:\n", response)


Response:
 Steve Jobs was the founder of Apple Inc. He conceived the name "Apple" after his return from India's Tantamount Hill (Apple Far M) in 1965. According to Walter Isaacson's biography, Steve Jobs took the idea for the company name to Cramer Electronics.

Response:
 The first product created by Apple mentioned in the provided context is the Apple I, which had only 200 units built. This pioneering personal computer served as a major success despite its limited production, powering Apple's rapidly growing parent company.

Response:
 The Apple logo was originally designed under a tree as part of the company's early branding efforts. Later on, the design evolved to feature a bite taken out of an Apple. These distinct logos are believed to be the reason for various theories surrounding the meaning behind the name "Apple." Some individuals speculate that the name Apple originates from these iconic logo designs, although this origin story remains uncertain.
Goodbye!
